In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import pyforest
import missingno as msno


#test statistique : 
from scipy.stats import spearmanr, kendalltau,pearsonr, shapiro

#import outils reductions dimensions
from sklearn.decomposition import PCA #linéaire
from sklearn.manifold import TSNE #non linéaire pour structure locale




#package modele non supervisé
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer #permet tracer courbe pour choisir bonne valeur KMeans
from sklearn.cluster import KMeans



import os
import warnings
warnings.filterwarnings("ignore")

- 
#### L'objectif de ce défi est de créer des modèles d'apprentissage automatique qui utilisent des données d'émissions de source ouverte (provenant des observations du satellite Sentinel-5P) pour prédire les émissions de carbone.

* **7 features principales** ont été extraites de janvier 2019 à novembre 2022. 
* Chaque caractéristique (dioxyde de soufre, monoxyde de carbone, etc.) **contient des sous caractéristiques** telles que
    - column_number_density qui est la densité verticale de la colonne au niveau du sol

- **SulphurDioxide** : pénètre dans l'atmosphère terrestre par des processus naturels et anthropiques.
- **CarbonMonoxide** : Dans certaines zones urbaines, il constitue un polluant atmosphérique majeur
- **NitrogenDioxide** : NO2 est utilisé pour représenter les concentrations d'oxydes d'azote collectifs car pendant la journée, c'est-à-dire en présence de lumière solaire, un cycle photochimique impliquant l'ozone (O3) convertit NO en NO2 et vice-versa sur une échelle de temps de quelques minutes.
- **Formaldehyde** : Le formaldéhyde est un gaz intermédiaire dans presque toutes les chaînes d'oxydation des composés organiques volatils non méthaniques (COVNM), qui aboutissent finalement au CO2.
- **UvAerosolIndex** :l'indice d'aérosol UV (UVAI), également appelé indice d'aérosol absorbant (AAI).Lorsque l'IAA est positif, il indique la présence d'aérosols absorbant les UV, tels que la poussière et la fumée.
- **Ozone** : Dans la stratosphère, la couche d'ozone protège la biosphère des dangereux rayonnements solaires ultraviolets. Dans la troposphère, elle agit comme un agent nettoyant efficace, mais à forte concentration, elle devient également nocive pour la santé de l'homme, des animaux et de la végétation. C'est un important gaz à effet de serre qui contribue au changement climatique
- **Cloud** : Ce jeu de données récupères les propriétés des nuages et fournit une imagerie hors ligne à haute résolution de ses paramètres.


# Plan

#### 1. Connaissance du jeu de données 
- Présence d'une target ?
- Features signification
- Dimensions des données
- Type de données et cohérence (variables discrètes : int, continues : float, qualitative : str)

#### 2. imputation des données :
- NaN présent : vrai NaN ou signifie une absence d'élément ?
- Visualisation des NaN
- Variables numériques :
    - discrètes : remplacement par le mode 
    - continues : moyenne ou médiane en fonction des outliers
    
### 3. Analyse univariée

### 4. Analyse bivariée & test statistiques
- Target(continue)vs features continues : 
    - heatmap
    - Test shapiro wilk
    - test de corrélation : Spearman/Kendall


### 5. Analyse multivariée
- Analyse des features continues via heatmap
- Analyse des variables corrélées et anticorrélées avec scatterplot

### 6. PCA & tSNE : sur l'ensemble du jeu de données (cad toutes les features, mais tester sur le train set uniquement)
#### PCA
- Standardisation des données
- Visualisation des données numériques
#### tSNE
- Visualisation des données numériques

### 7. PCA & tSNE : sur chaque groupe de features principale (uniquement sur le train set)
- Comme il y a beaucoup d'observation, on pourra faire un KMeans pour afficher les indivius moyen plutot que d'afficher toutes les observations

### 8. Preprocessing & Enregistrement des données 



# 1. Decouverte du jeu de données

In [ ]:
path = r"D:\Etude_Data_science\Kaggle_competition\04_Predict CO2 Emissions in Rwanda\dataset\train.csv"

In [ ]:
data = pd.read_csv(path)
df = data.copy()
pd.set_option("display.max_columns",None)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

- Avec le head et tail, on peut noter la présence de données manquantes, que le jeu de données continent des informations temporelles (on démarre de l'année 2019 au premier weekend jusqu'en 2021 au weekend 52, soit 53 semaines)

In [ ]:
pd.set_option("display.max_columns",10)

In [ ]:
df.info()

In [ ]:
#Aperçu rapide de quelques mesures statistiques
df.describe()
#on note la présence de données manquantes (count < nombre de ligne total du jeu de données)

In [ ]:
#Résumer du type de variable présent : 
df.dtypes.value_counts()

In [ ]:
#récupération de la target : 
target = df.emission

In [ ]:
#récupérons l'id du jeu de données 
identifiant = df.ID_LAT_LON_YEAR_WEEK

In [ ]:
#Verifions si l'id est unique pour chaque ligne :
identifiant.nunique() == df.shape[0]

## Premier bilan des observations : 
- La target est **emission** qui est une variable quantitative continues : **Regression**
- Il n'y a que des **variables numériques** : 
    * 2 discrètes (année : de 2019 a 2021 et le numéro de la semaine (allant de 0 pour la 1ere semaine à 52)
    * 73 continues : correspondent aux **features principales et leurs déclinaisons**
    * 1 ordinale : correspond à l'id de chaque lignes 
- Présence de données manquantes
- La nomenclature utilisé ici pour les noms de colonnes est **NomFeaturePrincipale_NomFeatureSecondaire** ce qui va nous permettre de créer une fonction qui facilitera la fraction du jeu de données pour pouvoir analyser plus simplement les 75 colonnes

In [ ]:
#Création d'une fonction permettant de lister toutes les colonnes selon un préfix définit : 
def find_columns(mask,dataframe):
    """Fonction qui renvoie une liste contenant toutes les colonnes de df commençant par le mask précisé en paramètre"""
    list_col = [i for i in dataframe if i[:len(mask)] == mask]
    return list_col

In [ ]:
# Exemple d'utilisation : Les colonnes avec carbone : 
col_carbon = find_columns("Carbon",df)
df[col_carbon].head() #Toutes les colonnes démarrant avec le mot clef "Carbon" seront récupérées 

Avec cette fonction, nous pourrons plus facilement accéder aux différentes colonnes du jeu de données, sachant que nous avons 7 nom de features principales, nous pourrons afficher plus facilement les colonnes par groupe de features principales

## 2. imputation des données :
NaN présent : vrai NaN ou signifie une absence d'élément ?

Comme nous somme sur des variables numériques, 0 représente une quantitée nulle.

Pour vérifier cela, regardons simplement si nous avons la présence de la valeur 0 dans le jeu de données. 

- **Cas 1 : Il n'y a pas de 0 présent dans les colonnes contenant des NaN ET dans les colonnes sans NaN** 
    - Il y a de forte chance que le NaN soit réellement un NaN et qu'il n'y a tout simplement pas de 0
- **Cas 2 : Il y a  des 0 présent dans les colonnes contenant des NaN ET dans les colonnes sans NaN** 
    - Forte chance que le NaN soit un NaN, la présence de 0 confirme une quantitée nulle, et le NaN lorsque l'info est manquante
    
- **Cas 3 : on comptabilise des 0 dans les colonnes contenant des NaN**
    - S'il y a des 0 en plus des NaN, alors le NaN est une donnée manquante
- **Cas 4 : Il n'y a pas de 0 uniquement dans les colonnes contenant des NaN**
    - S'il n'y a aucun 0 dans les colonnes contenant des NaN, alors qu'il y a des 0 dans les colonnes ou il n'y a pas de NaN, alors on est pratiquement sûr que les NaN correspondent a 0


In [ ]:
#Récuperons les colonnes contenant des NaN
col_with_nan = [i for i in df if df[i].isnull().sum()>0]
#Colonne sans NaN :
col_no_nan = [i for i in df if i not in col_with_nan]

print("Nombre de colonnes avec des NaN :", len(col_with_nan))
print("Nombre de colonnes sans NaN :", len(col_no_nan))


In [ ]:
#Regardons s'il y a des 0 pour les colonnes contenant au moins 1 NaN :
for i in df[col_with_nan]:
    compteur_0 = 0
    col = df[i]
    #somme des 0 de la colonne 
    cumsum = (col == 0).sum()
    if cumsum>0:#si on a au moins un 0 dans la colonne on incrémente le compteur
        compteur_0 +=1
    if compteur_0>0: #si le compteur a été incrémenter (donc si ya eu des 0), on l'affiche
        print(f" {i:-<35} : {compteur_0}")
    

In [ ]:
#Regardons le nombre de 0 pour les colonnes sans NaN : 

for i in df[col_no_nan]:
    compteur_0 = 0
    col = df[i]
    #somme des 0 de la colonne 
    cumsum = (col == 0).sum()
    if cumsum>0:#si on a au moins un 0 dans la colonne on incrémente le compteur
        compteur_0 +=1
    if compteur_0>0: #si le compteur a été incrémenter (donc si ya eu des 0), on l'affiche
        print(f" {i:-<35} : {compteur_0}")


### Bilan : 
* Pour les colonnes qui ont des NaN, certaines colonnes ont des 0
* Pour les colonnes qui n'ont pas de NaN, même observation

- Nous somme dans le cas 2, donc ici les NaN sont de vraies données manquantes que nous allons imputer

### Visualisation des NaN :

In [ ]:
msno.dendrogram(df[col_with_nan])
plt.show()

- Nous voyons la proximité des données manquantes, les colonnes partageant le même préfix sont les plus proches entre elles. Par exemple toutes les colonnes commençant par "NitrogenDioxide" partagent beaucoup plus de lignes communes avec des NaN que les colonnes commençant par "Uv" 

In [ ]:
msno.bar(df[col_with_nan], color="slateblue",sort='descending', )
plt.show()

Nous voyons ici qu'il y a quelques de colonnes contenant beaucoup de données manquantes, nous allons, en fonction de la quantité de données manquantes appliquer différentes stratégies : 

- **1 :** données avec moins de 40%, on impute
- **2 :** données avec plus de 40% de NaN on supprime colonne

In [ ]:
df.isnull().mean()[round(df.isnull().mean()*100,2)>40]

In [ ]:
df[col_with_nan].isnull().mean()[round(df[col_with_nan].isnull().mean()*100,2)<=40]

a faire :
- supprr colonne >40 en rcupérant le nom des colonnes
- recup colonne <40 faire limputation par moyenne ou mediane en faisant visualisation

In [ ]:
col_inf_40 = list(df[col_with_nan].isnull().mean()[round(df[col_with_nan].isnull().mean()*100,2)<=40].keys())
col_sup_40 = list(df[col_with_nan].isnull().mean()[round(df[col_with_nan].isnull().mean()*100,2)>40].keys())

In [ ]:
# Suppression des colonnes contenant trop de données manquantes : 
df.drop(col_sup_40, axis = 1, inplace=True)

In [ ]:
len(col_inf_40)

#### Imputation des données 
- Visualisation des outliers par boxplot :

In [ ]:
#Création d'une fonction pour visualiser plusieurs boxplot simultanément 
def multi_boxplot(columns, nrow, ncol):
    fig, axes = plt.subplots(nrow, ncol, figsize=(16, 12))
    for i, k in enumerate(columns):
        row = i // ncol
        col = i % ncol  # Correction ici
        ax = sns.boxplot(data=df, y=k, ax=axes[row, col], color ="skyblue")
        ax.set_ylabel('')# Suppression du ylabel
        ax.set_xlabel(k, rotation = 0,fontsize=6, fontweight='bold') #on le met plutot sur l'axe x
    plt.show()  # plt.show() en dehors de la boucle for pour afficher tous les boxplots en une seule fois


In [ ]:
multi_boxplot(col_inf_40[:20],4,5)

In [ ]:
multi_boxplot(col_inf_40[20:40],4,5)

In [ ]:
multi_boxplot(col_inf_40[40:],5,5)

#### Détection d'outliers  par méthode de calcul :

In [ ]:
def detecter_outliers(column):
    """Fonction qui calcul la quantité d'outlier d'une colonne donnée"""
    Q1 = column.quantile(0.25) #récupération des quantiles Q1 Q3
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1  #Calcul interquantile
    limite_inferieure = Q1 - 1.5 * IQR #on définit les bornes inférieur et supérieur
    limite_superieure = Q3 + 1.5 * IQR
    
    #Somme des outliers
    nombre_outliers = ((column < limite_inferieure) | (column > limite_superieure)).sum()
    
    #Calcul du % d'outliers sur la colonne
    pourcentage_outliers = (nombre_outliers / len(column)) * 100
    

    #renvoi du % arrondie au 10e
    return round(pourcentage_outliers,2)

In [ ]:
for i in col_inf_40:
    # Utilisation de la fonction pour détecter le pourcentage d'outliers dans une colonne
    pourcentage_outliers = detecter_outliers(df[i])
    if pourcentage_outliers>5:
        print(f"% Outlier {i} : {pourcentage_outliers}%")


- hormis pour 1 features, il n'y a pas de features présentant un % outliers > 5
- On va remplacer par la **moyenne** pour chaque variable

In [ ]:
#Imputation des données par Imputer :
imputer = SimpleImputer(strategy="mean")
imputer.fit(df[col_inf_40])

In [ ]:
#transformation des données : 
imputed_data = imputer.transform(df[col_inf_40])
#remplacement des données imputées dans le jeu de données  :
df[col_inf_40] = pd.DataFrame(imputed_data, columns=col_inf_40)

In [ ]:
#Vérifions s'il n'y a plus de données manquantes dans le jeu de données :
df.isnull().sum()[df.isnull().sum()>0]

## 3. Analyse univariée :

- Nous avons en quelque sorte démarré cette étape lors de l'imputation en affichant des boxplots.
- Nous pouvons aussi regarder la distribution des données  
    - Affichons la distribution
    - A l'aide d'un pairplot nous regarderons la distribution des données 
    - Nous regarderons la distribution de la target 

In [ ]:
def multi_distrib(columns, nrow, ncol):
    fig, axes = plt.subplots(nrow, ncol, figsize=(20, 16))
    for i, k in enumerate(columns):
        row = i // ncol
        col = i % ncol  # Correction ici
        ax = sns.distplot(a=df[k], ax=axes[row, col], color ="pink", kde_kws={"color":"red"})
        ax.axvline(df[k].mean(), label = "mean", ls="--",color = "orange")
        ax.axvline(df[k].median(),ls= ":", label = "median", color = "blue")
        ax.legend()
#         ax.set_xlabel('')# Suppression du xlabel
#         ax.set_ylabel(k, rotation = 0,fontsize=6, fontweight='bold') #on le met plutot sur l'axe x
    plt.show()  # Déplacez ce plt.show() en dehors de la boucle for pour afficher tous les boxplots en une seule fois


In [ ]:
col1 = df.columns[1:20]  #démarre par 1 pour exclure la premiere colonne qui est l'ID
col2 = df.columns[20:40]
col3 = df.columns[40:60]
col4 = df.columns[60:]

In [ ]:
multi_distrib(col1,5,4)

In [ ]:
multi_distrib(col2,5,4)

In [ ]:
multi_distrib(col3,5,4)

In [ ]:
multi_distrib(col4,3,3)

Au vu de la quantité d'observation, nous allons réduire notre jeu de données pour faciliter sa visualisation
- prenons 1000observations de celui ci

In [ ]:
df_frac = df.sample(n=1000,random_state=42)
df_frac.shape

In [ ]:
sns.pairplot(data=df_frac[col1])

In [ ]:
sns.pairplot(data=df_frac[col2])

In [ ]:
sns.pairplot(data=df_frac[col3])

In [ ]:
sns.pairplot(data=df_frac[col4])

- La distribution des données entre la target et les quelques features sur le 4è pairplot est **non linéaire**, donc lors du heatmap il y a de forte chance qu'on observe de faible corrélation entre la target et les différentes features

- Ensuite, on peut voir qu'il y a quelque features corrélées entre elles.


#### variables discrètes :
- weekno & year sont les deux variables discrètes et n'ont pas besoin de visualisation, il n'y a qu'un weekend par ligne et que 3 années différentes équitablement réparties

In [ ]:
year_x = df.year.value_counts().keys()
year_y = df.year.value_counts().values
sns.barplot(data = df, x = year_x , y=year_y)
plt.show()

#### Analyse target :

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(target, color='skyblue', edgecolor='black', bins = 20)
plt.xlabel('Emission :')
plt.ylabel('Fréquence')
plt.title('Distribution de la variable cible : Emission')
plt.grid(True)
plt.show()

On peut observer une distribution multimodal avec un premier pic majoritaire (frequence >60.000) suivi d'un pic  ~ 10.000 et enfin un dernier pic notable  ~ 100

In [ ]:
# Calcul de l'histogramme de la target
occurence, bin_edges = np.histogram(target, bins=20)  #mettons 20 bins comme sur la figure

#Affichons les 3 premier intervalle pour nos 3 modes principaux : 
print("Limites des 3 premiers bins : ")
print(bin_edges[:3])
print()
print("Nombre d'occurence pour les 3 premiers bins :")
print(occurence[:3])

### Analyse bivariée : 

-  Nous avons vu qu'il y avait peu de corrélation linéaire lorsque la target était impliquée.
- Nous allons faire une heatmap et regarder la corrélation linéaire de plus près entre la target et les différentes features
- Si, effectivement nous apercevons très peu de corrélation linéaire, nous devrons utiliser non pas le test de pearson mais celui de **Spearman** (préférable au test de Kendall au vu de la quantitée de données importantes ici)

In [ ]:
col1 = list(df.columns[1:20])  #démarre par 1 pour exclure la premiere colonne qui est l'ID
col2 = list(df.columns[20:40])
col3 = list(df.columns[40:60])
col4 = list(df.columns[60:])

In [ ]:
df = df.drop(identifiant.name, axis= 1)

In [ ]:
# Sélection des variables continues de votre jeu de données
var_cont = list(df.drop(["week_no","year","emission"], axis = 1).columns)
# Calcule de la corrélation entre la target et chaque variable continue
correlation = df[var_cont ].corrwith(df['emission']) #corrwith() permet de calculer la corrélation entre
# Trier les valeurs de corrélation par ordre décroissant
sorted_correlation = correlation.sort_values(ascending=False)
#la target et chaque variable continue
plt.figure(figsize=(10, 20))
heatmap = sns.heatmap(pd.DataFrame(sorted_correlation), annot=True, cmap='RdYlGn',fmt=".2%", cbar=True)
heatmap.set_title('Corrélation entre la target et les variables continues')
plt.show()

Nous confirmons notre hypothèse précédente, à savoir **qu'il y a une faible corrélation linéaire entre notre target et nos features**

- Nous allons faire un test de shapiro wilk et en fonction du résultat nous allons utiliser le test statistique adapté : 
    - Les features suivant une loi normal : on fera le test de **pearson**
    - Les features ne suivant pas une loi normal : on fera le test de **Kendall** (et non spearman car on a dit ici qu'il y avait beaucoup de données)

In [ ]:
### Fonction pour vérifier si une variable suit une loi normale : 
def shapiro_test(list_col, dataframe):
    """
    Fonction shapiro test qui renvoie une liste 
    
    """
    if type(list_col) == str:
        list_col = [list_col]
    
    var_normal = []
    var_non_normal = []
    
    # Test de normalité de Shapiro-Wilk
    for i in list_col:
        statistic, p_value = shapiro(dataframe[i])
#         print("feature :", i)
    
        # Afficher les résultats
#         print(f"Statistique de test W : {round(statistic,2)}")
#         print(f"P-value : {p_value}")
    
    # Interpréter les résultats
        alpha = 0.05  # Niveau de signification
    
        if p_value < alpha:
#             print("Les données ne suivent probablement pas une distribution normale.")
            var_non_normal.append(i)
        else:
#             print("Les données semblent suivre une distribution normale.")
            var_normal.append(i)
#         print()    
    
#     print("Features suivant une loi normale :", var_normal)
#     print("Features ne suivant pas une loi normale :", var_non_normal)
    return var_normal, var_non_normal

In [ ]:
#On va créer une liste contenant l'ensemble des features a l'exception de la target
all_features = [i for i in  list(df.columns) if i !=  target.name]

In [ ]:
var_norm, var_non_norm = shapiro_test(all_features, dataframe=df)

In [ ]:
print("Features suivant une loi normale : \n", var_norm)
print()
print("Features ne suivant pas une loi normale :\n", var_non_norm)

Aucune variable ne suit une loi normale donc nous allons utiliser uniquement le test de kendall :

In [ ]:
def kendall_test(feature, target, dataframe):
    """
    Fonction pour effectuer le test de corrélation de Kendall entre une feature et une target.
    
    Paramètres :
    - feature : Liste de features ou une feature unique
    - target : Nom de la target
    - dataframe : DataFrame contenant les données
    
    Retourne :
    - var_a_conserver : Liste des features ayant une corrélation significative avec la target
    - var_a_suppr : Liste des features n'ayant pas une corrélation significative avec la target
    """
    var_a_conserver = []
    var_a_suppr = []

    # Test de corrélation de Kendall
    for feat in feature:
        corr, p_value = kendalltau(dataframe[feat], dataframe[target])

        # Afficher les résultats
#         print(f"Feature : {feat}")
#         print(f"Corrélation avec {target} : {corr}")
#         print(f"P-value : {p_value}")

        # Interpréter les résultats
        alpha = 0.05  # Niveau de signification
        if p_value < alpha:
            var_a_conserver.append(feat)
        else:
            var_a_suppr.append(feat)
#         print()

#     print("Features à conserver :", var_a_conserver)
#     print("Features à supprimer :", var_a_suppr)
    return var_a_conserver, var_a_suppr


In [ ]:
var_a_conserver, var_a_supprimer  =kendall_test(all_features, "emission", dataframe=df)

In [ ]:
print("Liste des features à conserver : \n",  var_a_conserver)
print()
print("Liste des features à supprimer : \n",  var_a_supprimer)

In [ ]:
df = df.drop(var_a_supprimer, axis = 1)

In [ ]:
df.shape

In [ ]:
target.name

In [ ]:
#Récupération des variables discrètes
var_dis = [i for i in df if (i not in var_cont) & (i != target.name)]
var_dis

# PCA & tSNE : Sur le train set
- Au vu des données, nous avons remarqué qu'il y avait peu de linéarité lors des pairplot, ainsi que lors de la heatmap , nous allons donc en plus de faire une PCA, une tSNE afin d'avoir une meilleur appréciation des données.

- Pour cela, nous allons demarrer avec une PCA, nous ferons le cercle des corrélation et nous utiliserons aussi ce cercle pour nous aider à interpréter les données que nous obtiendrons après une réduction par tSNE

In [ ]:
X,y = df.drop(["emission"], axis = 1), df["emission"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print("Dimensions des données :\n")
print('X_train :', X_train.shape)
print('y_train :', y_train.shape)
print()
print('X_test :', X_test.shape)
print('y_test :', y_test.shape)

### Standardisation des données :

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[var_cont])

In [ ]:
#Réduction des données :
X_train_reduced = pd.DataFrame(scaler.transform(X_train[var_cont]), columns=var_cont, index = X_train.index)
X_test_reduced = pd.DataFrame(scaler.transform(X_test[var_cont]), columns=var_cont, index = X_test.index)

In [ ]:
#Concaténation de l'ensemble des features réduites : 
X_reduced = pd.concat([X_train_reduced, X_test_reduced])

In [ ]:
X_reduced.head()

In [ ]:
df_reduced = pd.concat([X_reduced,df[var_dis],y], axis = 1)

### PCA : Sur l'ensemble du jeu de données (cad sur toutes les colonnes, mais sur le train set)

In [ ]:
pca = PCA(n_components=10, random_state=42) #gardons les 10 premieres composantes

In [ ]:
x_pca = pca.fit_transform(X_train_reduced[var_cont])

In [ ]:
#Transformons les données pca directement en un dataframe
#ou les colonnes sont les components principales 
#et les lignes les individus 
df_x_pca = pd.DataFrame(x_pca, columns=[f"F{i+1}" for i in range(pca.n_components_)],index=X_train_reduced.index)
df_x_pca.head()

* Pour rappel les components principales sont le résultat de l'expression de chacune des features de bases
* F1 provient de l'ensemble de la somme des produits des features initiales. 
#### Nous y reviendrons ultérieurement


In [ ]:
#Recuperons les indices des principals components
n_components = [i for i in range(pca.n_components_)]

In [ ]:
#Récupération de la variance (autrement dit l'information conservée au travers chaque components principales)

#Calcul de la variance cumulée en %
sse = pca.explained_variance_ratio_*100
#Calcul de la somme cumulée de chaque variance :
sse_cum = [round(i,2) for i in np.cumsum(sse)]
sse_cum

### Regardons graphiquement ce que chaque composantes principale apporte comme information

In [ ]:
plt.figure(figsize=(8,4))
sns.lineplot(x = n_components, y=sse_cum, markers="d", color = "red",style=True)
ax = sns.barplot(x = n_components, y=sse,)
ax.get_legend().remove()#suppression de la legende pour le markers du lineplot
ax.set_xticklabels([i+1 for i in n_components])#pour commencer a compter à partir de 1 sur le graph
plt.grid()



In [ ]:
print(f"On passe de {pca.n_features_} à {pca.n_components_} features")
print(f"soit une réduction de {pca.n_features_-pca.n_components_} variables ")
print(f"Nous perdons {round((100-sse_cum[-1]))} % d'informations")

#### Regardons la relation entre les principal components et les features de bases :

In [ ]:
df_pcs = pd.DataFrame(data = pca.components_, 
                   columns=X_reduced.columns,
                  index = [f"F{i+1}" for i in range(pca.n_components_)]
                  )
                  
df_pcs.head()

In [ ]:
#inversons le dataframe pour faciliter la lisibilité: 
df_pcs = df_pcs.T
df_pcs.head()

In [ ]:
#Affichons une heatmap pour améliorer le confort de lecture : 

plt.figure(figsize=(20,20))
plt.title("Relation entre les variables synthéthique et les variables initiales",fontdict={"color":"red"})
sns.heatmap(pcs.T.sort_values(by="F1", ascending=False), annot = True, linecolor="black", linewidths=0.5, cmap="RdYlGn")
plt.show()

- La heatmap nous permet de voir les relations entre les variables initiales et les variables synthéthiques. 

* Comme dit plus haut  avec le dataframe **df_x_pca** ou nous avons les individus en lignes et en colonnes les nouvelles composantes.
- Pour mieux visualiser la relation entre les composantes principales et les variables initiales, nous pouvons tracer le cercle des corrélation : 

- Pour mieux visualiser la relation entre les composantes principales et les variables initiales, nous pouvons tracer le cercle des corrélation : 

### Cercle des corrélation : 

In [ ]:
def correlation_graph(pca, 
                      x_y, 
                      features) : 
    """Affiche le graphe des correlations
    Positional arguments : 
    -----------------------------------
    pca : sklearn.decomposition.PCA : notre objet PCA qui a été fit
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    features : list ou tuple : la liste des features (ie des dimensions) à représenter
    """
    # Extrait x et y 
    x,y=x_y
    # Taille de l'image (en inches)
    fig, ax = plt.subplots(figsize=(10, 9))
    # Pour chaque composante : 
    for i in range(0, pca.components_.shape[1]):
        # Les flèches
        ax.arrow(0,0, 
                pca.components_[x, i],  
                pca.components_[y, i],  
                head_width=0.07,
                head_length=0.07, 
                width=0.02, )
        # Les labels
        plt.text(pca.components_[x, i] + 0.05,
                pca.components_[y, i] + 0.05,
                features[i])      
    # Affichage des lignes horizontales et verticales
    plt.plot([-1, 1], [0, 0], color='grey', ls='--')
    plt.plot([0, 0], [-1, 1], color='grey', ls='--')

    # Nom des axes, avec le pourcentage d'inertie expliqué
    plt.xlabel('F{} ({}%)'.format(x+1, round(100*pca.explained_variance_ratio_[x],1)))
    plt.ylabel('F{} ({}%)'.format(y+1, round(100*pca.explained_variance_ratio_[y],1)))
    plt.title("Cercle des corrélations (F{} et F{})".format(x+1, y+1))
   
    # Le cercle 
    an = np.linspace(0, 2 * np.pi, 100)
    plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale

    # Axes et display
    plt.axis('equal')
    plt.show(block=False)

In [ ]:
correlation_graph(pca, (1,2), var_cont)

- On regarde le cercle des corrélation en parallèle avec la heatmap ci dessus pour mieux comprendre
- certaines variables partageant des caractéristique communes se trouvent dans les même direction (colorisée en  orange sur F1 de la heatmap)

### utilisation algorithme de clustering
- Nous allons utiliser un Kmeans afin de mieux voir les clusters avec de nouveaux label pour mieux apprecier les nouveaux clusters qui se dessinent (s'il y en a). Le but de faire ce clustering c'est de voir si on ne peut pas attribuer de nouveaux labels a ces individus.
- De plus, nous afficherons aussi le graphique des individus afin de voir si on peut interpréter les clusters d'individus

In [ ]:
def Kmeans_elbow(data, nclust):
    plt.figure(figsize=(8,4))
    visualizer = KElbowVisualizer(KMeans(), k=(1, nclust))  # Spécifiez la plage de k que vous souhaitez explorer
    visualizer.fit(data)
    visualizer.show()

In [ ]:
Kmeans_elbow(df_x_pca, 10)

On voit 3 clusters sur les données

In [ ]:
# Affichage des clusters avec colorisation en fonction de la target
sns.scatterplot(x=df_x_pca.iloc[:, 0], y=df_x_pca.iloc[:, 1], hue=y_train, palette="viridis")
plt.title('Clusters colorisés en fonction de la target (sans kmeans)')
plt.xlabel('F1')
plt.ylabel('F2')
plt.show()

- On voit ici que les 2 premiers plans sont mal capturé, en effet on se retrouve avec un nuage de point très dense ne laissant apparaitre aucun cluster.
- La qualité d'information, la quantité de features ainsi que le nombre d'observation qui rend toutes ces relations complexes, non linéaire sont a l'origine de cela.

In [ ]:
def graph_kmeans(data, cluster,x,y):
    kmeans = KMeans(n_clusters=cluster, random_state=42)  
    kmeans.fit(data)  

# labels de cluster attribués à chaque point de données
    labels = kmeans.labels_

#attribuons un nom pour les labels de la sur graphique : 
    lab_graph = [F"Cluster {i}" for i in labels]
    plt.title("Données colorisées après un kmeans")
    sns.scatterplot(x=data.iloc[:, x], y=data.iloc[:, y],
                hue=lab_graph, #hue pour donner le nom des clusters
                hue_order=np.unique(lab_graph), #l'ordre dans lequel on veut afficher les noms grâce à la légende
                palette="bright")
    plt.legend(bbox_to_anchor=(1,1))
    plt.xlabel(f"F{x}")
    plt.ylabel(f"F{y}")
    plt.show()


In [ ]:
graph_kmeans(data=df_x_pca, cluster=3,x=0,y=1)


- On peut voir ici une grosse difficulté a séparer les clusters, toutes les données sont mélangées
- C'était attendu, en effet, nous savons que nos données présentent des relations complexes entre elles avec peut de linéarité, nous avons des données complexes, ainsi, il serait peut être plus pertinent d'utiliser une réduction dimensionnalité avec un tSNE

### tSNE : Sur l'ensemble des features du jeu de données (mais train set)
- Attention cet algorithme est gourmand et nécessite demande beaucoup plus de temps s'il y a un nombre important de données

In [ ]:
tsne = TSNE(n_components=2, random_state=42)

In [ ]:
X_tsne = tsne.fit_transform(X_train_reduced[var_cont])

In [ ]:
Kmeans_elbow(X_tsne, 6)

In [ ]:
# Affichage des clusters avec colorisation en fonction de la target
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=y_train, palette="viridis")
plt.title('Clusters colorisés en fonction de la target (sans kmeans)')
plt.xlabel('F1')
plt.ylabel('F2')
plt.show()

In [ ]:
graph_kmeans(data=X_tsne, cluster=3,x=0,y=1)


- La séparation des clustersr est légèrement améliorée, surtout entre le cluster 0 et 1 mais ca reste encore complexe
- la PCA et le tSNE ne sont pas convaincant, tentons une autre approche 

### 8. PCA & tSNE : En fonction de chaque features principales

In [ ]:
print( "Nous avons en tout", df_reduced.shape[1] ,"variables")

Ces variables sont catégorisée selon la hiérarchie suivante suivante : Une feature principale est accompagnée de X sous features.
- Nous avons 7 features principales :

In [ ]:
df_reduced


- **SulphurDioxide** : pénètre dans l'atmosphère terrestre par des processus naturels et anthropiques.
- **CarbonMonoxide** : Dans certaines zones urbaines, il constitue un polluant atmosphérique majeur
- **NitrogenDioxide** : NO2 est utilisé pour représenter les concentrations d'oxydes d'azote collectifs car pendant la journée, c'est-à-dire en présence de lumière solaire, un cycle photochimique impliquant l'ozone (O3) convertit NO en NO2 et vice-versa sur une échelle de temps de quelques minutes.
- **Formaldehyde** : Le formaldéhyde est un gaz intermédiaire dans presque toutes les chaînes d'oxydation des composés organiques volatils non méthaniques (COVNM), qui aboutissent finalement au CO2.
- **UvAerosolIndex** :l'indice d'aérosol UV (UVAI), également appelé indice d'aérosol absorbant (AAI).Lorsque l'IAA est positif, il indique la présence d'aérosols absorbant les UV, tels que la poussière et la fumée.
- **Ozone** : Dans la stratosphère, la couche d'ozone protège la biosphère des dangereux rayonnements solaires ultraviolets. Dans la troposphère, elle agit comme un agent nettoyant efficace, mais à forte concentration, elle devient également nocive pour la santé de l'homme, des animaux et de la végétation. C'est un important gaz à effet de serre qui contribue au changement climatique
- **Cloud** : Ce jeu de données récupères les propriétés des nuages et fournit une imagerie hors ligne à haute résolution de ses paramètres.

récupérons ces différentes features principales et leurs sous features dans différentes variables

In [ ]:
dioxyde_soufre = find_columns("Sulphu",df)
monoxyde = find_columns("Carbo",df)
nitrogen = find_columns("Nitrogen",df)
formaldehyde = find_columns("Formaldehyde",df)
uvaerosol = find_columns("UvAerosolIndex",df)
ozone = find_columns("Ozone",df)
cloud = find_columns("Cloud",df)


all_type_col = [dioxyde_soufre, monoxyde, nitrogen, formaldehyde, uvaerosol, ozone,cloud]
dict_main_col = {"dioxyde_soufre":dioxyde_soufre,
               "monoxyde":monoxyde,
               "nitrogen":nitrogen,
               "formaldehyde":formaldehyde,
               "uvaerosol":uvaerosol,
               "ozone":ozone,
               "cloud":cloud}

In [ ]:
all_type_col[:2]

In [ ]:
len_col = [len(i) for i in all_type_col]
sum(len_col) #il manque 5 colonnes

#### Repérons les colonnes manquantes : 
- Pour cela, nous allons deja faire une seule liste, car le probleme avec notre variable all_type_col c'est que c'est une liste qui contient des sous liste de chaque features qui démarre par le préfix donné par notre fonction find_columns
- Or nous voulons faire une seule liste qui contiendra toutes les features, pour cela nous allons l'"aplatir" 

In [ ]:
liste_aplatie = []
# Aplatissons notre liste en ajoutant chaque élément des sous-listes à la liste aplatie
for sous_liste in all_type_col:
    liste_aplatie.extend(sous_liste)

liste_aplatie[:5] #nous voyons que nous n'avons plus de sous liste


Regardons quelles features est passé entre les mailles du filet :

In [ ]:
[i for i in df if i not in liste_aplatie] #on regarde les colonnes qui sont sur df mais qu'on a pas su récupérer 
#avec l'utilisation de notre fonction find_columns

- heuresement ! Ce sont les features qui ne font pas parti des 7 features principales, nous avons la target, les coordonnées GPS, l'année et le week-end

* Pour nos 7 ACP, nous allons pouvoir utiliser chacune des variables contenant l'ensemble des sous features.
- Exemple avec dioxyde_soufre = find_columns("Sulphu",df), nous allons faire une ACP contenant toutes les features avec le soufre

### L'objectif est de faire autant d'ACP pour chaque feature principale afin de réduire le nombre de dimension (de feature) puis ensuite de concaténer toutes ces features ensemble afin d'avoir un jeu de donnée réduit.
### Cela rendra la démarche plus lisible que notre premiere tentative ou nous avons fait une ACP directement sur tous le jeu de données !

Nous pourrons visualiser le cercle des corrélation ainsi que le graphique des individus pour chaque ACP

In [ ]:
#récupération des noms que nous avons donné pour chaque feature principale
name_main_feat = [i for i in dict_main_col.keys()]
name_main_feat 

In [ ]:
dict_main_col[name_main_feat[0]]

In [ ]:
def instant_columns(indice, voir_liste=False):
    """
    fonction qui permet d'acceder directement a l'ensemble des sous
    features d'une features principal en indexant une valeur contenue dans 
    notre liste des noms de features principales
    """
    if voir_liste:
        print(name_main_feat)
    
    taille_ind = len(name_main_feat)
    if indice not in range(0,len(name_main_feat)):
        print("l'indice doit se situer entre ",min(range(taille_ind)), "et", max(range(taille_ind)))
    return dict_main_col[name_main_feat[indice]]

Exemple d'utilisation :
- on affiche la liste contenant le nom des features principal
- Le 1er element de la liste est dioxyde_soufre, donc si on met l'indice 0 on affichera tous les noms de sous features contenant "SulphurDioxide" 

In [ ]:
instant_columns(0,True)

## 8. PCA 
- Faisons une PCA pour les 7 features principales

#### 1er groupe de feature principale

In [ ]:
def quick_pca(indice, n_components=0.8):
    """
    indice : int ; 
    fonction  qui renvoi un tuple de deux dataframe et de la pca :
    1er dataframe contient :
    les individus en ligne et les principal components en colonne
    2e dataframe contient :
    Les principal components en ligne et les features initiale
    """
    #verification de la condition de l'indice
    taille_ind = len(name_main_feat)
    if indice not in range(0,len(name_main_feat)):
        print("l'indice doit se situer entre ",min(range(taille_ind)), "et", max(range(taille_ind)))
    
    #Mise ne place de la PCA : 
    pca = PCA(n_components=n_components, random_state = 42)
    x_pca = pca.fit_transform(X_train_reduced[instant_columns(indice)])
    #Récupération des indices de chaque composants principales Fn..
    n_components = [i for i in range(pca.n_components_)]

    #Création du dataframe individus/principale composantes
        
    df_x_pca = pd.DataFrame(x_pca, 
    columns=[f"F{i+1}_{z.split('_')[0]}"  #création d'un index unique sous la forme Fn_nom_feature_principal
                           for (i,z) in zip(range(pca.n_components_), instant_columns(indice)) ],
                            
    index=X_train_reduced.index
    )
    
    #Création du dataframe variables initiales/principale composantes
    
    df_pcs = pd.DataFrame(data = pca.components_, 
                   columns=instant_columns(indice),
                  index = [f"F{i+1}_{z.split('_')[0]}"  #création d'un index unique sous la forme Fn_nom_feature_principal
                           for (i,z) in zip(range(pca.n_components_), instant_columns(indice)) ]
                  )
    df_pcs = df_pcs.T
    return (df_x_pca, df_pcs, pca)
    

In [ ]:
def hm(pcs):
    # heatmap pour améliorer le confort de lecture : 

    plt.figure(figsize=(7,5))
    plt.title("Relation entre les variables synthéthique et les variables initiales",fontdict={"color":"red"})
    ax = sns.heatmap(pcs, annot = True, linecolor="black", fmt=".2", linewidths=0.5, cmap="RdYlGn")
    ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=45, fontsize=6)
    plt.show()

In [ ]:
def screeplot_pca(pca):
    
   #Récupération de la variance (autrement dit l'information conservée au travers chaque components principales)

#Calcul de la variance cumulée en %
    sse = pca.explained_variance_ratio_*100
    #Calcul de la somme cumulée de chaque variance :
    sse_cum = [round(i,2) for i in np.cumsum(sse)]
    
    #Récupération des indices de chaque composants principales Fn..
    n_components = [i for i in range(pca.n_components_)]
    
    sns.reset_defaults()
    #Tracer de figure :
    plt.figure(figsize=(8,4))
    plt.title(f"Graphique des éblouies des sommes cumulées de la variance")
    sns.lineplot(x = n_components, y=sse_cum, markers="o", color = "red",style=True)
        # Ajout des valeurs de sse_cum à chaque point du markers du lineplot
    for i, txt in enumerate(sse_cum):
        plt.text(n_components[i], sse_cum[i], f'{txt}%')

    #Tracer du barplot des sommes cumulées:
    ax = sns.barplot(x = n_components, y=sse)
    ax.get_legend().remove()#suppression de la legende pour le markers du lineplot
    ax.set_xticklabels([i+1 for i in n_components])#pour commencer a compter à partir de 1 sur le graph
    plt.grid()
    plt.show()
    print(f"On passe de {pca.n_features_} à {pca.n_components_} features")
    print(f"soit une réuction de {pca.n_features_-pca.n_components_} variables ")
    print(f"Nous perdons {round((100-sse_cum[-1]))} % d'informations")

In [ ]:
def pca_individus(df_pca):
    # Affichage des clusters avec colorisation en fonction de la target
    sns.scatterplot(x=df_pca.iloc[:, 0], y=df_pca.iloc[:, 1], hue=y_train, palette="viridis")
    plt.title('Clusters colorisés en fonction de la target (sans kmeans)')
    plt.xlabel('F1')
    plt.ylabel('F2')
    plt.show()

In [ ]:
df_sulphur_pca, df_sulphur_pcs, pca_sulphur = quick_pca(0)

In [ ]:
screeplot_pca(pca_sulphur)

In [ ]:
hm(df_sulphur_pcs)

In [ ]:
correlation_graph(pca_sulphur,(0,1), instant_columns(0))

In [ ]:
Kmeans_elbow(X_train_reduced[instant_columns(0)], nclust=5)

In [ ]:
graph_kmeans(df_sulphur_pca, 3, 0,1)

#### 2e groupe de feature principale

In [ ]:
df_carbon_pca, df_carbon_pcs, pca_carbon = quick_pca(1)

In [ ]:
screeplot_pca(pca_carbon)
hm(df_carbon_pcs)
correlation_graph(pca_carbon,(0,1), instant_columns(1))
Kmeans_elbow(X_train_reduced[instant_columns(1)], nclust=5)
graph_kmeans(df_carbon_pca, 3, 0,1)

#### 3e groupe de feature principale

In [ ]:
df_nitrogen_pca, df_nitrogen_pcs, pca_nitrogen = quick_pca(2)

In [ ]:
screeplot_pca(pca_carbon)
hm(df_carbon_pcs)
correlation_graph(pca_carbon,(0,1), instant_columns(1))
Kmeans_elbow(X_train_reduced[instant_columns(1)], nclust=5)
graph_kmeans(df_carbon_pca, 3, 0,1)

#### 4e groupe de feature principale

In [ ]:
df_formaldehyde_pca, df_formaldehyde_pcs, pca_formaldehyde = quick_pca(3)

In [ ]:
screeplot_pca(pca_carbon)
hm(df_carbon_pcs)
correlation_graph(pca_carbon,(0,1), instant_columns(1))
Kmeans_elbow(X_train_reduced[instant_columns(1)], nclust=5)
graph_kmeans(df_carbon_pca, 3, 0,1)

#### 5e groupe de feature principale

In [ ]:
df_uv_pca, df_uv_pcs, pca_uv = quick_pca(4)

In [ ]:
screeplot_pca(pca_carbon)
hm(df_carbon_pcs)
correlation_graph(pca_carbon,(0,1), instant_columns(1))
Kmeans_elbow(X_train_reduced[instant_columns(1)], nclust=5)
graph_kmeans(df_carbon_pca, 3, 0,1)

#### 6e groupe de feature principale

In [ ]:
df_ozone_pca, df_ozone_pcs, pca_ozone = quick_pca(5)

In [ ]:
screeplot_pca(pca_carbon)
hm(df_carbon_pcs)
correlation_graph(pca_carbon,(0,1), instant_columns(1))
Kmeans_elbow(X_train_reduced[instant_columns(1)], nclust=5)
graph_kmeans(df_carbon_pca, 3, 0,1)

#### 7e groupe de feature principale

In [ ]:
df_cloud_pca, df_cloud_pcs, pca_cloud = quick_pca(6)

In [ ]:
screeplot_pca(pca_carbon)
hm(df_carbon_pcs)
correlation_graph(pca_carbon,(0,1), instant_columns(1))
Kmeans_elbow(X_train_reduced[instant_columns(1)], nclust=5)
graph_kmeans(df_carbon_pca, 3, 0,1)